In [1]:
from nltk.util import pr
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.model_selection import train_test_split
# from sklearn.model_selection import cross_val_score
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
import re
import nltk
stemmer = nltk.SnowballStemmer("english")
from nltk.corpus import stopwords
import string
nltk.download('stopwords')
stopword=set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/alex/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data = pd.read_csv("./dataset/labeled_data.csv")

In [3]:
pd.set_option('display.max_colwidth', None)
data = data.rename(columns={'Unnamed: 0': 'idx'})

In [4]:
data.head()

,idx,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she look like a tranny
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361;


## data preproccessing


In [5]:
tweets = data[["tweet"]]
tweets

,tweet
0,!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...
1,!!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she look like a tranny
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361;
...,...
24778,"you's a muthaf***in lie &#8220;@LifeAsKing: @20_Pearls @corey_emanuel right! His TL is trash &#8230;. Now, mine? Bible scriptures and hymns&#8221;"
24779,"you've gone and broke the wrong heart baby, and drove me redneck crazy"
24780,young buck wanna eat!!.. dat nigguh like I aint fuckin dis up again
24781,youu got wild bitches tellin you lies


In [6]:
def clean(text):
    text = str(text).lower()
    text = re.sub('', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [7]:
data["tweet"] = data['tweet'].apply(clean)

In [8]:
def preprocessing (text):
    text = [word for word in text.split() if word not in stopword]
    text = " ".join(text)
    text = [stemmer.stem(word) for word in text.split()]
    text = " ".join(text)
    return text


In [9]:
data["tweet"] = data['tweet'].apply(clean)

In [10]:
data.head()

,idx,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,rt mayasolovely as a woman you shouldnt complain about cleaning up your house amp as a man you should always take the trash out
1,1,3,0,3,0,1,rt boy dats coldtyga dwn bad for cuffin dat hoe in the place
2,2,3,0,3,0,1,rt urkindofbrand dawg rt you ever fuck a bitch and she start to cry you be confused as shit
3,3,3,0,2,1,1,rt cganderson vivabased she look like a tranny
4,4,6,0,6,0,1,rt shenikaroberts the shit you hear about me might be true or it might be faker than the bitch who told it to ya


In [11]:
class_new = data["class"].map({0:1,1:1,2:0})
data["class"] = class_new
data.head()

,idx,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,0,rt mayasolovely as a woman you shouldnt complain about cleaning up your house amp as a man you should always take the trash out
1,1,3,0,3,0,1,rt boy dats coldtyga dwn bad for cuffin dat hoe in the place
2,2,3,0,3,0,1,rt urkindofbrand dawg rt you ever fuck a bitch and she start to cry you be confused as shit
3,3,3,0,2,1,1,rt cganderson vivabased she look like a tranny
4,4,6,0,6,0,1,rt shenikaroberts the shit you hear about me might be true or it might be faker than the bitch who told it to ya


In [12]:
categories = {2:"robot hate-speech", 1:"human hate-speech",0:"another",}
value = data['class'].map(categories)

In [13]:
index = data.columns.get_loc("tweet")
data.insert(index,"value",value)

In [14]:
data = data.drop(['count', 'hate_speech', 'offensive_language', 'neither'], axis=1)

In [15]:
data.head()

,idx,class,value,tweet
0,0,0,another,rt mayasolovely as a woman you shouldnt complain about cleaning up your house amp as a man you should always take the trash out
1,1,1,human hate-speech,rt boy dats coldtyga dwn bad for cuffin dat hoe in the place
2,2,1,human hate-speech,rt urkindofbrand dawg rt you ever fuck a bitch and she start to cry you be confused as shit
3,3,1,human hate-speech,rt cganderson vivabased she look like a tranny
4,4,1,human hate-speech,rt shenikaroberts the shit you hear about me might be true or it might be faker than the bitch who told it to ya


In [16]:
filtered_data = data[data['class'] == 1]

# Map the 'class' values to their corresponding categories
filtered_data['category'] = filtered_data['class'].map(categories)

# Print the filtered data with the mapped categories
print(filtered_data)

         idx  class              value  \
1          1      1  human hate-speech   
2          2      1  human hate-speech   
3          3      1  human hate-speech   
4          4      1  human hate-speech   
5          5      1  human hate-speech   
...      ...    ...                ...   
24776  25289      1  human hate-speech   
24777  25290      1  human hate-speech   
24778  25291      1  human hate-speech   
24780  25294      1  human hate-speech   
24781  25295      1  human hate-speech   

                                                                                                                    tweet  \
1                                                          rt  boy dats coldtyga dwn bad for cuffin dat hoe in the  place   
2                            rt urkindofbrand dawg rt  you ever fuck a bitch and she start to cry you be confused as shit   
3                                                                          rt cganderson vivabased she look like a trann

/tmp/ipykernel_12506/720204218.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['category'] = filtered_data['class'].map(categories)


In [17]:
file_path = 'hate_dataset.json'


filtered_data.to_json(file_path, orient='records', indent=4)
# # Write the optimal_values list to a JSON file
# with open(file_path, 'w') as json_file:
#     json.dump(optimal_values_list, json_file)

# print("Optimal values saved to:", file_path)

In [22]:
import csv
csv_file = 'generated_data.csv'
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Tweet", "Model Output"])  # Write header
    for index, row in filtered_data.iterrows():
        writer.writerow([row['tweet'], ''])

print(f"CSV file generated: {csv_file}")

CSV file generated: generated_data.csv
